Logic:
- Có 1 file ggshet là Arrival (những SKU đã về kho)
- Có 1 file ggsheet book DS (book tồn trên AMZ - chỉ cần điền ttin, bot tự động book trên AMZ) (sheet "DS booking")
=> Lấy những con đã về kho nhưng chưa book DS (SKU mới, chưa có trên sheet DS Booking hoặc SKU hết hàng, SIC tự book tay, không điền vào DS booking) => Alert

In [1]:
# Create DataFrame
import re
import numpy as np
import pandas as pd
# Visualize
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import seaborn.objects as so
# Connect Portgresat
import psycopg2
# Connect MySQL
import mysql.connector
import pyodbc
# Connect Google Sheet
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from googleapiclient.discovery import build
from google.oauth2.service_account import Credentials
# Time
#from datetime import datetime, timedelta
from datetime import datetime, timedelta
import datetime
import time
import schedule
# System
import os
import sys
import pickle

#C:/Users/hieptn/Desktop/Setup/Python/instantclient_21_10
import warnings
warnings.filterwarnings('ignore') # os.chdir(r'C:\Connection')
from sqlalchemy import create_engine
import sqlalchemy.exc
# Format Mail
from pretty_html_table import build_table
from bs4 import BeautifulSoup
#MAil
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.application import MIMEApplication

import numpy as np
from tabulate import tabulate
import textwrap

In [2]:
# Load the key file for the service account
credentials = ServiceAccountCredentials.from_json_keyfile_name(r"G:\Shared drives\Deployment Data Template\Power BI\Python Code\2.Daily\email-sending-via-python-fd13e6126e20.json",
scopes=['https://www.googleapis.com/auth/chat.bot'])

# Build the service object
chat = build('chat', 'v1', credentials=credentials) #v1 cho chat bot, v4 cho edit ggshet

# Lấy data trong room space chat
spaces = chat.spaces().list().execute()
members = chat.spaces().members().list(parent=spaces['spaces'][2]['name']).execute() # đổi id để lấy space cần thiết
members1 = members['memberships']

result = []
for item in members1:
    # Extract the member name and display name from the nested dictionary
    member_data = item.get('member', {})
    member_name = member_data.get('name', 'N/A')  # Sử dụng giá trị mặc định 'N/A' nếu không tìm thấy 'name'
    display_name = member_data.get('displayName', 'N/A')  # Sử dụng giá trị mặc định 'N/A' nếu không tìm thấy 'displayName'
    # Add the extracted information into a new dictionary
    result.append({'member_name': member_name, 'display_name': display_name})

df_sales_pic_chat = pd.DataFrame(result)

# Lấy thông tin Sales PIC
# Cấu hình thông tin xác thực và kết nối đến Google Sheets API
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name(r"G:\Shared drives\Deployment Data Template\Power BI\Python Code\2.Daily\email-sending-via-python-fd13e6126e20.json", scope)
client = gspread.authorize(creds)
service = build('sheets', 'v4', credentials=creds) #v1 cho chat bot, v4 cho edit ggshet
# Lấy dữ liệu từ Google Sheet 1 
spreadsheet_id1 = '1L7kZhRNkojHPqdQOHVYEdD9OYKdFw843JkRjMvh5HgI'
#https://docs.google.com/spreadsheets/d/1L7kZhRNkojHPqdQOHVYEdD9OYKdFw843JkRjMvh5HgI/edit#gid=0
range_name = 'Sales PIC!A1:C'
result = service.spreadsheets().values().get(
        spreadsheetId=spreadsheet_id1, range=range_name).execute()
values = result.get('values', [])
df_sales_pic = pd.DataFrame(result['values'][1:], columns=result['values'][0])

In [3]:
spaces

{'spaces': [{'name': 'spaces/AAAAeR-aZjc',
   'type': 'ROOM',
   'displayName': 'Test chat bot',
   'spaceThreadingState': 'THREADED_MESSAGES',
   'spaceType': 'SPACE',
   'spaceHistoryState': 'HISTORY_ON',
   'createTime': '2024-01-10T08:01:47.610553Z'},
  {'name': 'spaces/AAAAq6c6dr0',
   'type': 'ROOM',
   'displayName': 'DL Alert OOB Bot',
   'spaceThreadingState': 'THREADED_MESSAGES',
   'spaceType': 'SPACE',
   'spaceDetails': {'description': 'Warn bots about OOB, Ineligible Hourly'},
   'spaceHistoryState': 'HISTORY_ON',
   'createTime': '2023-04-22T12:36:52.436835Z'},
  {'name': 'spaces/AAAA1aINor8',
   'type': 'ROOM',
   'displayName': 'Test Alert (Stock onsite status/ DS Booking SKUs)',
   'spaceThreadingState': 'THREADED_MESSAGES',
   'spaceType': 'SPACE',
   'spaceDetails': {'description': 'Test'},
   'spaceHistoryState': 'HISTORY_ON',
   'createTime': '2023-06-06T11:06:56.381804Z'}]}

In [4]:
df_sales_pic_chat

,member_name,display_name
0,users/101441363125954665107,Thiên Nguyễn Hoàng
1,users/111715880194110953044,Giang Đỗ Thu
2,users/115457224128174038130,Thành Khấu Nguyễn Trường
3,users/104831017180979964601,Yến Lâm Kim
4,users/110033644358569229509,Khôi Nguyễn Kim
5,users/114480486560260912816,Vỹ Lương Minh
6,users/101591789831969138242,Hùng Võ Trí


## Tạo hàm

In [3]:
def Alert_DS_Booking():   
    #### Data BI (Quang Ù)
    conn_post_1 = psycopg2.connect(host="172.29.15.98",database="mi_report", user="da_report",password="Inno202$")

    #### Lấy list Asin, SKU
    list_asin_sku = pd.read_sql_query(f"""
        select asin, sku from public.launching_plan where asin is not null and sku is not null """,conn_post_1)
    #### List_asin
    list_asin = tuple(list_asin_sku['asin'])
    #### List_sku
    list_sku = tuple(list_asin_sku['sku'])

    # Cấu hình thông tin xác thực và kết nối đến Google Sheets API
    scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
    creds = ServiceAccountCredentials.from_json_keyfile_name(r"G:\Shared drives\Deployment Data Template\Power BI\Python Code\2.Daily\email-sending-via-python-fd13e6126e20.json", scope)
    client = gspread.authorize(creds)
    service = build('sheets', 'v4', credentials=creds)

    # Lấy dữ liệu từ Google Sheet INVENTORY UPDATE DAILY
    spreadsheet_id = '1oxqHbj9MFnAqFtvcREoWN6Fp9p3aCNH1ZS8i9paZLbc'
    #https://docs.google.com/spreadsheets/d/1oxqHbj9MFnAqFtvcREoWN6Fp9p3aCNH1ZS8i9paZLbc/edit#gid=24679269
    range_name = 'INVENTORY!A:L'
    result = service.spreadsheets().values().get(
            spreadsheetId=spreadsheet_id, range=range_name).execute()
    values = result.get('values', [])
    df_inv_up_daily = pd.DataFrame(result['values'][1:], columns=result['values'][0])

    # Fillter list SKU
    df_inv_up_daily_1 = df_inv_up_daily[df_inv_up_daily['SKU'].isin(list_sku)]
    df_inv_up_daily_2 = df_inv_up_daily_1[df_inv_up_daily_1['INVENTORY SALABLE']!=""]
    df_inv_up_daily_3 = df_inv_up_daily_2[df_inv_up_daily_2['INVENTORY SALABLE'].notna()]

    list_sku_inv_daily = df_inv_up_daily_3[['SKU']]

    list_sku_inv_daily['SKU'] = list_sku_inv_daily['SKU'].str.strip()

    # Lấy dữ liệu từ Google Sheet Deployment - DS Booking
    spreadsheet_id_1 = '1SbGA5kxIPjNSNDohn0AsxBl2s_QawwI0sNMzOvjbwQs'
    #https://docs.google.com/spreadsheets/d/1SbGA5kxIPjNSNDohn0AsxBl2s_QawwI0sNMzOvjbwQs/edit#gid=627736277
    range_name_1 = 'DS booking!A2:M'
    result_1 = service.spreadsheets().values().get(
            spreadsheetId=spreadsheet_id_1, range=range_name_1).execute()
    values_1 = result_1.get('values', [])
    df_ds_booking = pd.DataFrame(result_1['values'][1:], columns=result_1['values'][0])

    # Fillter list SKU
    list_sku_DS_Booking = df_ds_booking[['SKU']]
    list_sku_DS_Booking['SKU'] = list_sku_DS_Booking['SKU'].str.strip()
    
    list_info_sku = pd.read_sql_query(f"""
    select SALES_LEAD, SALES_PIC, SKU, ASIN, CATEGORY, PRODUCT_GROUP, VARIATIONS from public.launching_plan
    where asin is not null and sku is not null
    """,conn_post_1)
    list_info_sku.columns = list_info_sku.columns.str.upper()

    # SKU Alert
    T_alert = list_sku_inv_daily[~list_sku_inv_daily['SKU'].isin(list_sku_DS_Booking['SKU'])]
    
    df_alert = T_alert.merge(list_info_sku, left_on ='SKU', right_on= 'SKU', how='left')

    df_alert_1 = df_alert.rename(columns= {'SALES_LEAD': 'Sales Lead','SALES_PIC': 'Sales PIC','CATEGORY': 'Category',
                                   'PRODUCT_GROUP': 'Product Group', 'VARIATIONS': 'Product Name'})

    df_alert_2 =df_alert_1[['Sales Lead','SKU','Category','Product Name']]

    # Check table
    if df_alert.empty:
        print("DataFrame DS Booking is empty!")
        print("Job executed at", datetime.datetime.now())
    else:
        markdown_table = tabulate(df_alert_2, headers='keys', tablefmt="grid", showindex=False,
                                  maxcolwidths=[8, 5, 20, 35],
                                  colalign=["left", "left", "right", "right"])

        header = f"⚠️*DS Booking Warning:*⚠️"

        text = f"""❗<users/110033644358569229509>, <users/101441363125954665107> ơi! Các SKU này chưa có trong DS Booking (<https://docs.google.com/spreadsheets/d/1SbGA5kxIPjNSNDohn0AsxBl2s_QawwI0sNMzOvjbwQs/edit#gid=0|Link>) work với Sales Lead điền vào nhé.
                """

        result = chat.spaces().messages().create(parent= 'spaces/AAAA1aINor8', #'spaces/AAAAq6c6dr0'                                            
                                                 body={"text": f"""{header}

{text}```{markdown_table}```""",}
                                                ).execute()   

        print("Job executed at", datetime.datetime.now())
        print("Alert Thành Công")

In [4]:
schedule.every().day.at("09:00").do(Alert_DS_Booking)
while True:
    schedule.run_pending()
    time.sleep(10)

Job executed at 2024-02-22 09:00:09.818024
Alert Thành Công


In [13]:
Alert_DS_Booking()

Job executed at 2024-02-21 09:28:09.584923
Alert Thành Công


# Test + Lấy data only

## Lấy list SKU

In [ ]:
#### Data BI (Quang Ù)
conn_post_1 = psycopg2.connect(host="172.29.15.98",database="mi_report", user="da_report",password="Inno202$")

#### Lấy list Asin, SKU
list_asin_sku = pd.read_sql_query(f"""
    select asin, sku from public.launching_plan where asin is not null and sku is not null """,conn_post_1)
#### List_asin
list_asin = tuple(list_asin_sku['asin'])
#### List_sku
list_sku = tuple(list_asin_sku['sku'])

In [ ]:
# Cấu hình thông tin xác thực và kết nối đến Google Sheets API
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name("C:/Users/hieptn/Desktop/Test Py/my-project-123-378311-71b37cb67041.json", scope)
client = gspread.authorize(creds)
service = build('sheets', 'v4', credentials=creds)

## Lấy INVENTORY UPDATE DAILY

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [ ]:
# Lấy dữ liệu từ Google Sheet 1
spreadsheet_id = '1oxqHbj9MFnAqFtvcREoWN6Fp9p3aCNH1ZS8i9paZLbc'
range_name = 'INVENTORY!A:P'
result = service.spreadsheets().values().get(
        spreadsheetId=spreadsheet_id, range=range_name).execute()
values = result.get('values', [])
df_inv_up_daily = pd.DataFrame(result['values'][1:], columns=result['values'][0])

In [ ]:
df_inv_up_daily

In [ ]:
# Fillter list SKU
df_inv_up_daily_1 = df_inv_up_daily[df_inv_up_daily['SKU'].isin(list_sku)]
df_inv_up_daily_2 = df_inv_up_daily_1[df_inv_up_daily_1['INVENTORY SALABLE']!=""]
df_inv_up_daily_3 = df_inv_up_daily_2[df_inv_up_daily_2['INVENTORY SALABLE'].notna()]

list_sku_inv_daily = df_inv_up_daily_3[['SKU']]

list_sku_inv_daily['SKU'] = list_sku_inv_daily['SKU'].str.strip()

In [ ]:
list_sku_inv_daily

## Lấy INV Yes4all

In [ ]:
df_inv_up_daily_3

In [ ]:
df_inv_y4a = df_inv_up_daily_3[['SKU','INVENTORY NASA SELLABLE','INVENTORY ANAHEIM SELLABLE']]
df_inv_y4a_1 = pd.melt(df_inv_y4a, id_vars=['SKU'], var_name='WH', value_name='value')
df_inv_y4a_2 = df_inv_y4a_1[df_inv_y4a_1['value']!=""]
df_inv_y4a_3 = df_inv_y4a_2[df_inv_y4a_2['value'].notna()]

In [ ]:
df_inv_y4a_3

## Deployment - DS Booking

In [ ]:
# Lấy dữ liệu từ Google Sheet 2
#spreadsheet_id_1 = '1SbGA5kxIPjNSNDohn0AsxBl2s_QawwI0sNMzOvjbwQs'
range_name_1 = 'DS booking!A2:M'
result_1 = service.spreadsheets().values().get(
        spreadsheetId=spreadsheet_id_1, range=range_name_1).execute()
values_1 = result_1.get('values', [])
df_ds_booking = pd.DataFrame(result_1['values'][1:], columns=result_1['values'][0])

In [ ]:
df_ds_booking

In [ ]:
list_sku_DS_Booking = df_ds_booking[['SKU']]

list_sku_DS_Booking['SKU'] = list_sku_DS_Booking['SKU'].str.strip()

## List SKU + WH

In [ ]:
list_sku_DS_Booking_WH = df_ds_booking[['SKU',"warehouse"]].dropna()

In [ ]:
list_sku_DS_Booking_WH['Link Col'] = list_sku_DS_Booking_WH.apply(lambda row: row['SKU'] +'-'+ row['warehouse'], axis=1)

In [ ]:
list_sku_DS_Booking_WH

## Check Data 1

In [ ]:
T_alert = list_sku_inv_daily[~list_sku_inv_daily['SKU'].isin(list_sku_DS_Booking['SKU'])]

In [ ]:
T_alert

## Check data 2

In [ ]:
T_alert_2 = df_inv_y4a_4.merge(list_sku_DS_Booking_WH, left_on ='Link Col', right_on= 'Link Col', how='left')

In [ ]:
T_alert_3 = T_alert_2[T_alert_2['warehouse'].isna()]

In [ ]:
T_alert_3

## Info Sku

In [ ]:
list_info_sku = pd.read_sql_query(f"""
select SALES_LEAD, SALES_PIC, SKU, ASIN, CATEGORY, PRODUCT_GROUP, VARIATIONS from BI_DATAMART_1.launching_plan
where asin is not null and sku is not null
""",db)

In [ ]:
list_info_sku

In [ ]:
df_alert = T_alert.merge(list_info_sku, left_on ='SKU', right_on= 'SKU', how='left')

In [ ]:
df_alert

In [ ]:
df_alert_1 = df_alert.rename(columns= {'SALES_LEAD': 'Sales Lead','SALES_PIC': 'Sales PIC','CATEGORY': 'Category',
                                       'PRODUCT_GROUP': 'Product Group', 'VARIATIONS': 'Product Name'})

In [ ]:
df_alert_2 =df_alert_1[['Sales Lead','SKU','Category','Product Name']]

In [ ]:
markdown_table = tabulate(df_alert_2, headers='keys', tablefmt="grid", showindex=False,
                          maxcolwidths=[8, 5, 20, 50],
                          colalign=["left", "left", "right", "right"])

In [ ]:
print(markdown_table)